# Setup

In [ ]:
!pip install ir_datasets -q
!pip install nltk -q
!pip install ir_measures -q
!pip install PyStemmer -q
!pip install pandas -q
!pip install python-terrier -q
!pip install --upgrade gdown -q

In [ ]:
import ir_datasets
import ir_measures
from ir_measures import *
import random
import re
import string
import nltk
import time
from collections import Counter, defaultdict
from tqdm.auto import tqdm
import gzip
import pickle
import os
import heapq
import math
import pyterrier as pt
from google.colab import drive
import os
import shutil

In [ ]:
import time

def profile(f):
    def f_timer(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        end = time.time()
        ms = (end - start) * 1000
        print(f"{f.__name__} ({ms:.3f} ms)")
        return result
    return f_timer

In [ ]:
dataset = ir_datasets.load("msmarco-passage")

# Mount Google Drive to access required files
drive.mount('/content/drive')

# # URL of the Google Drive repository containing the project files
# repository = "YOUR_REPOSITORY_URL"
# repository_name = "ir-project-files"

# # Download the specified folder from the repository
# !gdown --folder $repository

# # Copy the downloaded files from the repository folder to /content/
# # This ensures the files are easily accessible during execution
# for item in os.listdir(repository_name):
#   s = os.path.join(repository_name, item)
#   d = os.path.join('/content/', item)
#   if os.path.isfile(s):             # Check if the item is a file before copying
#     shutil.copy2(s, d)

# # Remove the downloaded repository folder to free up space
# shutil.rmtree(repository_name)

# Preprocessing

In [ ]:
from functools import lru_cache
import Stemmer
nltk.download("stopwords", quiet=True)

# ------- Pre Initialization -------
# Compile reusable resources and pre-load common datasets for efficiency
# 1. Regular expression for removing unnecessary dots in acronyms
# 2. Translation table for stripping punctuation
# 3. Set of English stopwords for filtering irrelevant tokens
# 4. Initialize a stemming tool for word normalization

ACRONYM_REGEX = re.compile(r"(?<!\w)\.(?!\d)")                  # Matches dots not part of decimal numbers
PUNCTUATION_TRANS = str.maketrans("", "", string.punctuation)   # Removes punctuation
STOPWORDS = set(nltk.corpus.stopwords.words('english'))         # Load English stopwords
STEMMER = Stemmer.Stemmer('english')                            # Initialize an English stemmer
# ----------------------------------

def preprocess(s):
    """
    Preprocesses an input string for text analysis tasks such as indexing or querying.

    Args:
        s (str): The input string to preprocess.

    Returns:
        list[str]: A list of processed tokens.
    """

    s = s.lower()
    s = s.replace("&", " and ")
    s = s.translate(str.maketrans("‘’´“”–-", "'''\"\"--"))      # Standardize quotes and dashes for uniformity
    s = ACRONYM_REGEX.sub("", s)                                # Remove unnecessary dots in acronyms (but not decimals)
    s = s.translate(PUNCTUATION_TRANS)                          # Remove all punctuation
    s = " ".join(s.split())                                     # Remove extra spaces and strip leading/trailing spaces

    tokens = s.split()
    tokens = [t for t in tokens if t not in STOPWORDS]          # Filter out stopwords
    tokens = STEMMER.stemWords(tokens)                          # Apply stemming to normalize word forms
    return tokens

# Build the inverted index

In [ ]:
@profile
def build_index(dataset):
    """
    Constructs an inverted index from a dataset.

    The function processes documents to build the following components:
    1. Lexicon: Maps terms to term IDs and tracks document frequency (DF) and term frequency (TF).
    2. Inverted Index: Maps term IDs to lists of document IDs and term frequencies.
    3. Document Index: A list of document IDs and their corresponding document lengths.
    4. Index Statistics: A dictionary summarizing the index statistics.

    Args:
        dataset: The dataset to index.

    Returns:
        tuple: A tuple containing:
            - lexicon (dict): Maps terms to [term ID, document frequency, term frequency].
            - inverted_index (dict): Contains:
                - 'docids' (dict): Maps term IDs to lists of document IDs.
                - 'freqs' (dict): Maps term IDs to lists of term frequencies in the documents.
            - document_index (list): A list of tuples (document ID, document length).
            - stats (dict): Contains:
                - 'num_docs': Total number of documents indexed.
                - 'num_terms': Total number of unique terms.
                - 'num_tokens': Total number of tokens across all documents.
    """

    lexicon = {}                # Maps terms to [term ID, document frequency, term frequency]
    doc_index = []              # Stores document IDs and their lengths
    inv_d, inv_f = {}, {}       # Inverted index components: doc IDs and term frequencies
    termid = 0                  # Counter for assigning unique term IDs

    num_docs = 0                # Number of documents processed
    total_dl = 0                # Total length of the documents (in tokens)

    # Iterate over documents in the dataset
    for docid, doc in tqdm(enumerate(dataset.docs_iter()), desc='Indexing', total=dataset.docs_count()):
        tokens = preprocess(doc.text)               # Preprocess document text into tokens
        token_tf = Counter(tokens)                  # Count term frequencies in the document

        # Populate the lexicon and inverted index
        for token, tf in token_tf.items():          # Assign a new term ID if the token is not in the lexicon
            if token not in lexicon:
                lexicon[token] = [termid, 0, 0]
                inv_d[termid], inv_f[termid] =  [], []
                termid += 1

            token_id = lexicon[token][0]            # Get the term ID
            inv_d[token_id].append(docid)
            inv_f[token_id].append(tf)
            lexicon[token][1] += 1                  # Increment document frequency for the term
            lexicon[token][2] += tf                 # Increment total term frequency

        # Update document index and statistics
        doclen = len(tokens)
        doc_index.append((str(doc.doc_id), doclen)) # Add document ID and length to the index
        total_dl += doclen
        num_docs += 1

    # Build index statistics
    stats = {
        'num_docs': 1 + docid,                      # Total number of documents indexed
        'num_terms': len(lexicon),                  # Total number of unique terms
        'num_tokens': total_dl,                     # Total number of tokens across all documents
        "avg_length": total_dl / (1 + docid)
    }

    return lexicon, {'docids': inv_d, 'freqs': inv_f}, doc_index, stats

In [ ]:
lex, inv, doc, stats = None, None, None, None               # Initialize variables for the index components

files = ['lexicon.pickle.gz', 'inverted_file.pickle.gz', 'document_index.pickle.gz', 'stats.pickle.gz']
if all(os.path.exists(file) for file in files):             # Check if all required files exist
    print("All files already exist.")

    # Iterate over the list of files and their associated variable names
    for file, var_name in zip(files, ['lex', 'inv', 'doc', 'stats']):
        try:
            if os.path.getsize(file) > 0:                   # Ensure the file is not empty
                with gzip.open(file, 'rb') as f:
                    globals()[var_name] = pickle.load(f)    # Load the file into the corresponding variable
            else:
                print(f"Warning: {file} is empty.")
        except EOFError:
            # If the file is corrupted or incomplete, rebuild the index
            print(f"Error: {file} is corrupted or incomplete. Rebuilding the index.")
            lex, inv, doc, stats = build_index(dataset)
            break
else:
    # If any of the files do not exist, rebuild the index
    lex, inv, doc, stats = build_index(dataset)

    # Save the rebuilt index components back into the respective files
    for data, file in zip([lex, inv, doc, stats], files):
      with gzip.open(file, 'wb') as f:
        print(f"Saving {file}...")
        pickle.dump(data, f)                                # Serialize and save the data


## Compression

Compression lexicon:

*   delta encoding per l'array
*   front coding per i termini



In [ ]:
def front_coding_encode(terms):
    compressed = []
    prev = terms[0]
    compressed.append(prev)
    for curr in terms[1:]:
        i = 0
        while i < min(len(prev), len(curr)) and prev[i] == curr[i]:
            i += 1
        suffix = curr[i:]
        compressed.append(f"{i}|{suffix}")
        prev = curr
    return compressed

def front_coding_decode(front_coded):
  terms = [front_coded[0]]
  for item in front_coded[1:]:
      prefix_len, suffix = item.split("|", 1)
      prefix_len = int(prefix_len)
      term = terms[-1][:prefix_len] + suffix
      terms.append(term)
  return terms

def delta_encode(arr):
    if not arr:
        return []
    encoded = [arr[0]]
    for i in range(1, len(arr)):
        encoded.append(arr[i] - arr[i-1])
    return encoded

def delta_decode(arr):
    if not arr:
        return []
    decoded = [arr[0]]
    for i in range(1, len(arr)):
        decoded.append(decoded[-1] + arr[i])
    return decoded

In [ ]:
terms = sorted(lex.keys())

front_coded_terms = front_coding_encode(terms)

# Extract metadata arrays aligned with sorted terms
term_ids = [lex[t][0] for t in terms]
dfs = [lex[t][1] for t in terms]
tfs = [lex[t][2] for t in terms]

# Delta encode metadata
term_ids_delta = delta_encode(term_ids)
dfs_delta = delta_encode(dfs)
tfs_delta = delta_encode(tfs)

# Compress metadata
term_ids_comp = compress_metadata(term_ids_delta)
dfs_comp = compress_metadata(dfs_delta)
tfs_comp = compress_metadata(tfs_delta)

pickle.dumps({
        "front_coded_terms": front_coded_terms,
        "term_ids_comp": term_ids_comp,
        "dfs_comp": dfs_comp,
        "tfs_comp": tfs_comp,
    })

In [ ]:
#!!!for the compression and decompression the RAM offered by colab is not sufficent
import pickle
import numpy as np
from tqdm import tqdm

class Compression:
    def __init__(self, inv):
        self.inv = inv

    @staticmethod
    def VBEncode(n):
        byte = []
        while True:
            byte.append(n % 128)
            if n < 128:
                break
            n //= 128
        byte[0] += 128  # Set the control bit on the first byte
        return byte[::-1]

    @staticmethod
    def VBEncodeList(n_list):
        b = bytearray()
        for n in n_list:
            b.extend(Compression.VBEncode(n))
        return b

    @staticmethod
    def VBDecode(byte_array):
        n_list = []
        n = 0
        for b in byte_array:
            if b < 128:
                n = 128 * n + b
            else:
                n = 128 * n + (b - 128)
                n_list.append(n)
                n = 0
        return n_list

    @staticmethod
    def dgaps(n_list):
        return [n_list[0]] + list(np.diff(n_list))

    def vb_compress_docids(self, doc_ids):
        # Apply delta gaps to docids and then encode
        return Compression.VBEncodeList(Compression.dgaps(doc_ids))

    def compress_inverted_index(self):
        compressed_inv = {'docids': {}, 'freqs': {}}
        for term_id, doc_ids in tqdm(self.inv['docids'].items(), desc="Compressing docids"):
            compressed_inv['docids'][term_id] = self.vb_compress_docids(doc_ids)  # Compress docids with delta gaps
            compressed_inv['freqs'][term_id] = self.inv['freqs'][term_id]  # Leave freqs unchanged
        return compressed_inv

    def vb_decompress_docids(self, byte_array):
        # Decode and then reconstruct original docids using cumulative sum
        dgaps = Compression.VBDecode(byte_array)
        return list(np.cumsum(dgaps))

    def decompress_inverted_index(self, compressed_inv):
        decompressed_inv = {'docids': {}, 'freqs': {}}
        for term_id, encoded_docids in tqdm(compressed_inv['docids'].items(), desc="Decompressing docids"):
            decompressed_inv['docids'][term_id] = self.vb_decompress_docids(encoded_docids)   # Decompress docids
            decompressed_inv['freqs'][term_id] = compressed_inv['freqs'][term_id]  # Retrieve freqs unchanged
        return decompressed_inv


In [ ]:
compression=Compression(inv)
compressed_inv = compression.compress_inverted_index()

In [ ]:
# Save the compressed index
import pickle
import humanize
import os

with open('comp_dict.pickle', 'wb') as f:
    pickle.dump((compressed_inv), f)

print(f'The compressed inverted index requires {humanize.naturalsize(os.path.getsize("comp_dict.pickle"))} bytes (all included)')

In [ ]:
decompression=Compression(compressed_inv)
inv=decompression.decompress_inverted_index(compressed_inv)

## Inverted index

In [ ]:
class InvertedIndex:
    """
    A simple inverted index class. Stores term-document mappings for fast retrieval.

    Attributes:
        lexicon (dict): Maps a token to [termID, docFreq, totalTermFreq].
        inv (dict): Contains 'docids' and 'freqs' lists, indexed by termID.
        doc (list): Each element is (doc_id, doc_length).
        stat (dict): Index statistics (e.g., num_docs, num_terms, num_tokens).

    Methods:
        num_docs() -> int
            Returns the total number of indexed documents.
        get_posting(termid: int) -> PostingListIterator
            Returns a posting list iterator for the given termID.
        get_termids(tokens: list[str]) -> list[int]
            Converts tokens to termIDs if found in the lexicon.
        get_postings(termids: list[int]) -> list[PostingListIterator]
            Returns posting list iterators for each termID.
    """

    class PostingListIterator:
        """
        (Inner class) Iterates over the posting list for a single termID.

        Attributes:
            docids (list[int]): Document IDs containing this term.
            freqs (list[int]): Term frequencies in the corresponding docID.
            pos (int): Current index in the posting list.
            doc (list): Reference to the main document index.

        Methods:
            docid() -> int or math.inf
                Returns the current docID or math.inf if finished.
            score() -> float or math.inf
                Returns freq / doc_length or math.inf if finished.
            next(target: int = None) -> None
                Moves forward or jumps to target docID if specified.
            is_end_list() -> bool
                Checks if the iterator has reached the end.
            len() -> int
                Returns the total number of docIDs for this term.
        """

        def __init__(self, docids, freqs, doc):
            """
            Initialize the iterator with document IDs, frequencies, and a reference to the document index.
            """
            self.docids = docids            # List of document IDs where the term appears
            self.freqs = freqs              # List of term frequencies corresponding to each document ID
            self.pos = 0                    # Start position in the posting list
            self.doc = doc                  # Reference to the main document index

        def docid(self):
            """
            Returns the current document ID or math.inf if the end of the list is reached.
            """
            if self.is_end_list():
                return math.inf
            return self.docids[self.pos]

        def score(self):
            """
            Computes the term frequency normalized by the document length for the current position.
            Returns math.inf if the end of the list is reached.
            """
            if self.is_end_list():
                return math.inf
            return self.freqs[self.pos]/self.doc[self.docid()][1]

        def next(self, target = None):
            """
            Advances to the next position in the posting list or jumps to the target document ID.
            """
            if not target:                              # If no target is specified, move to the next position
                if not self.is_end_list():
                    self.pos += 1
            else:
                if target > self.docid():               # If a target is specified, jump to its position if it exists
                    try:
                        self.pos = self.docids.index(target, self.pos)
                    except ValueError:
                        self.pos = len(self.docids)     # Move to the end if the target is not found

        def is_end_list(self):
            """
            Checks if the iterator has reached the end of the posting list.
            """
            return self.pos == len(self.docids)


        def len(self):
            """
            Returns the total number of document IDs in the posting list.
            """
            return len(self.docids)


    def __init__(self, lex, inv, doc, stats):
        """
        Initialize the inverted index with its components: lexicon, inverted file, document index, and stats.
        """
        self.lexicon = lex          # Lexicon mapping tokens to [termID, docFreq, totalTermFreq]
        self.inv = inv              # Inverted index with 'docids' and 'freqs'
        self.doc = doc              # List of documents with IDs and lengths
        self.stat = stats           # Index statistics (e.g., number of documents, terms, tokens)

    def num_docs(self):
        """
        Returns the total number of indexed documents.
        """
        return self.stats['num_docs']

    def get_posting(self, termid):
        """
        Returns a PostingListIterator for the given term ID.
        """
        return InvertedIndex.PostingListIterator(self.inv['docids'][termid], self.inv['freqs'][termid], self.doc)

    def get_termids(self, tokens):
        """
        Converts a list of tokens to their corresponding term IDs using the lexicon.
        """
        return [self.lexicon[token][0] for token in tokens if token in self.lexicon]

    def get_postings(self, termids):
        """
        Returns a list of PostingListIterators for the given term IDs.
        """
        return [self.get_posting(termid) for termid in termids]

inv_index = InvertedIndex(lex, inv, doc, stats)

# Query processing

In [ ]:
trec_dl_2020 = ir_datasets.load("msmarco-passage/trec-dl-2020")

In [ ]:
class TopQueue:
    """
    A simple top-k priority queue to maintain the top-scoring items.

    This class uses a min-heap to efficiently store and retrieve the top-k
    items based on their scores. Items are tuples of (score, docid).

    Attributes:
        queue (list[tuple[float, int]]): The priority queue storing (score, docid) pairs.
        k (int): The maximum number of items to maintain in the queue.
        threshold (float): The minimum score required for an item to enter the queue.

    Methods:
        size() -> int:
            Returns the current number of items in the queue.
        would_enter(score: float) -> bool:
            Checks if a given score exceeds the threshold and could enter the queue.
        clear(new_threshold: float = None) -> None:
            Clears the queue and optionally sets a new threshold.
        insert(docid: int, score: float) -> bool:
            Attempts to insert an item into the queue. Updates the threshold if needed.
        __repr__() -> str:
            Returns a string representation of the queue.
    """

    def __init__(self, k=10, threshold=0.0):
        """
        Initializes the TopQueue with a maximum size and an optional threshold.
        """
        self.queue = []                     # Initialize an empty priority queue (min-heap)
        self.k = k                          # Maximum number of items to store
        self.threshold = threshold          # Initial score threshold

    def size(self):
        """
        Returns the current number of items in the queue.
        """
        return len(self.queue)

    def would_enter(self, score):
        """
        Checks if a given score exceeds the current threshold and could enter the queue.
        """
        return score > self.threshold

    def clear(self, new_threshold=None):
        """
        Clears all items from the queue and optionally sets a new threshold.
        """
        self.queue = []                     # Empty the queue
        if new_threshold is not None:
            self.threshold = new_threshold  # Update the threshold if provided

    def __repr__(self):
        """
        Returns a string representation of the queue.
        """
        return f'<{self.size()} items, th={self.threshold} {self.queue}>'

    def insert(self, score, query_id, docid):
        # Inserts an item into the priority queue if it meets the threshold
        if not self.would_enter(score):
            return False  # The score is too low to be added to the heap

        # Adds the query ID along with the score and document ID
        if self.size() < self.k:
            heapq.heappush(self.queue, (score, query_id, docid))
        else:
            heapq.heapreplace(self.queue, (score, query_id, docid))

        # Updates the threshold only if the queue contains at least `k` elements
        if self.size() >= self.k:
            self.threshold = self.queue[0][0]

        return True

    def sort_descending(self):
        # Sorts the queue by score in descending order
        self.queue = sorted(self.queue, key=lambda x: x[0], reverse=True)
        return self.queue

## BM25

In [ ]:
LOG_E_OF_2 = math.log(2)        # Natural logarithm of 2 for base conversion
LOG_2_OF_E = 1 / LOG_E_OF_2     # Conversion factor for log base-e to base-2

# Compute average document length and total number of documents from the index
avg_dl = inv_index.stat['num_tokens'] / inv_index.stat['num_docs']
N = inv_index.stat['num_docs']

def bm25(tf, df, dl, k1=1.2, b=0.75):
    """
    Compute the BM25 relevance score for a term in a document.

    Args:
        tf (int):   Term frequency, the count of the term in the document.
        df (int):   Document frequency, the number of documents containing the term.
        dl (float): Document length, the number of tokens in the document.
        k1 (float): Parameter controlling term frequency saturation.
        b (float):  Parameter controlling document length normalization.

    Returns:
        float: The BM25 score for the term in the document with respect to the query.
    """
    idf = math.log(1 + (N - df + 0.5) / (df + 0.5)) * LOG_2_OF_E                # IDF weighting
    K = k1 * ((1 - b) + b * (dl / avg_dl))                                      # Document length adjustment
    term_frequency_component = ((k1 + 1) * tf) / (K + tf)                       # TF component
    return idf * term_frequency_component

### DAAT BM25

In [ ]:
# Precompute document lengths
doc_lengths = defaultdict(int)
for docid, doc_len in inv_index.doc:
    doc_lengths[docid] = doc_len

def min_docid(postings):
    """
    Find the smallest document ID among active posting list iterators.

    Args:
        postings (list[PostingListIterator]): Posting list iterators.

    Returns:
        int: The smallest document ID or math.inf if all lists are exhausted.
    """

    min_docid = math.inf
    for p in postings:
        if not p.is_end_list():     # Skip completed lists
            min_docid = min(p.docid(), min_docid)
    return min_docid

def daat_bm25(postings, k=10):
    """
    Perform Document-At-A-Time (DAAT) retrieval with BM25 scoring.

    Args:
        postings (list[PostingListIterator]): Posting lists for terms.
        k (int): Number of top results to retrieve.

    Returns:
        list[tuple[int, float]]: Top-k (docid, score) pairs sorted by score.
    """

    top = TopQueue(k)                               # Initialize top-k priority queue
    current_docid = min_docid(postings)             # Start with the smallest document ID

    while current_docid != math.inf:                # Process documents until all posting lists are exhausted
        score = 0
        next_docid = math.inf

        for posting in postings:
            if posting.docid() == current_docid:    # Check if the term is in the current doc
                tf = posting.freqs[posting.pos]
                df = posting.len()
                dl = doc_lengths[current_docid]

                score += bm25(tf, df, dl)

                posting.next()                      # Move to the next term occurrence

            if not posting.is_end_list():           # Update the smallest doc ID for next iteration
                next_docid = min(next_docid, posting.docid())

        top.insert(current_docid, score)            # Add the current doc to the top-k queue
        current_docid = next_docid                  # Move to the next document

    return sorted(top.queue, reverse=True)

### TAAT BM25

In [ ]:
def taat_bm25(postings, k=10):
    """
    Perform Term-At-A-Time (TAAT) retrieval with BM25 scoring.

    Args:
        postings (list[PostingListIterator]): A list of posting list iterators, one for each query term.
        k (int): The maximum number of top documents to retrieve. Default is 10.

    Returns:
        list[tuple[int, float]]: A sorted list of (docid, score) tuples, ordered by score in descending order.
    """
    A = defaultdict(float)                      # Accumulator for document scores

    # Process one term's posting list at a time
    for posting in postings:
        current_docid = posting.docid()
        df = posting.len()                      # Document frequency for the current term

        while current_docid != math.inf:
            tf = posting.freqs[posting.pos]     # Term frequency in the current document
            dl = doc_lengths[current_docid]     # Length of the current document

            score = bm25(tf, df, dl)            # Compute BM25 score for the term-document pair
            A[current_docid] += score

            posting.next()
            current_docid = posting.docid()

    top = TopQueue(k)

    for docid, score in A.items():              # Insert all documents and their scores into the top-k queue
        top.insert(docid, score)

    return sorted(top.queue, reverse=True)

## TF-IDF

In [ ]:
def tfidf_score(tf, df, dl, k1 = 1.2, b = 0.75):
    """
    Compute the TF-IDF score using a normalized term frequency formulation.

    Args:
        tf (int): Term frequency in the document.
        df (int): Document frequency, the number of documents containing the term.
        dl (float): Document length, the total number of tokens in the document.
        k1 (float): Term frequency saturation parameter.
        b (float): Length normalization parameter.

    Returns:
        float: The TF-IDF score for the term in the document with respect to the query.
    """
    # Compute normalized term frequency
    tf_robertson = k1 * tf / (tf + (k1 * ((1 - b) + ((b * dl) / avg_dl))))
    # Compute inverse document frequency (IDF) with base-2 logarithm
    idf = math.log((N / df) + 1) * LOG_2_OF_E

    return tf_robertson * idf

### DAAT TF-IDF

In [ ]:
def daat_tfidf(postings, k=10):
    """
    Perform Document-At-A-Time (DAAT) retrieval using TF-IDF scoring.

    Args:
        postings (list[PostingListIterator]): A list of posting list iterators, one for each query term.
        k (int): The maximum number of top documents to retrieve. Default is 10.

    Returns:
        list[tuple[int, float]]: A sorted list of (docid, score) tuples, ordered by score in descending order.
    """

    top = TopQueue(k)                               # Initialize a priority queue for the top-k results
    current_docid = min_docid(postings)             # Start with the smallest document ID across postings

    while current_docid != math.inf:                # Loop until all documents are processed
        score = 0
        next_docid = math.inf

        for posting in postings:
            if posting.docid() == current_docid:    # Check if the term appears in the current document
                tf = posting.freqs[posting.pos]
                df = posting.len()
                dl = doc_lengths[current_docid]

                score += tfidf_score(tf, df, dl)    # Accumulate the TF-IDF score for this document

                posting.next()

            if not posting.is_end_list():           # Update the next smallest document ID
                next_docid = min(next_docid, posting.docid())

        top.insert(current_docid, score)            # Insert the document and its score into the top-k queue
        current_docid = next_docid                  # Move to the next document to be scored

    return sorted(top.queue, reverse=True)

### TAAT TF-IDF

In [ ]:
def taat_tfidf(postings, k=10):
    """
    Perform Term-At-A-Time (TAAT) retrieval using TF-IDF scoring.

    Args:
        postings (list[PostingListIterator]): A list of posting list iterators, one for each query term.
        k (int): The maximum number of top documents to retrieve.

    Returns:
        list[tuple[int, float]]: A sorted list of (docid, score) tuples, ordered by score in descending order.
    """
    A = defaultdict(float)                      # Accumulator for document scores

    # Process one term's posting list at a time
    for posting in postings:
        current_docid = posting.docid()

        df = posting.len()

        while current_docid != math.inf:
            tf = posting.freqs[posting.pos]
            dl = doc_lengths[current_docid]

            score = tfidf_score(tf, df, dl)      # Compute TF-IDF score
            A[current_docid] += score

            posting.next()
            current_docid = posting.docid()

    top = TopQueue(k)

    for docid, score in A.items():              # Insert all documents and their scores into the top-k queue
        top.insert(docid, score)

    return sorted(top.queue, reverse=True)

# Results

In [ ]:
@profile
def query_processing(queries_iter, fn):
    """
    Process a list of queries using a specified scoring function.

    Args:
        queries_iter (iterable): An iterable of query objects.
        fn (callable): A scoring function that takes a list of posting list iterators
            and returns a list of (docid, score) tuples.

    Returns:
        list[dict]: A list of results, each containing:
            - `query_id` (int): The ID of the processed query.
            - `scores` (list[tuple[int, float]]): The list of (docid, score) tuples for the query.
    """

    res = []                                # Store the results for each query

    for q in queries_iter:
        query = preprocess(q.text)                  # Preprocess the query text
        termids = inv_index.get_termids(query)      # Map query tokens to term IDs
        postings = inv_index.get_postings(termids)  # Retrieve posting lists for the term IDs

        # Compute scores using the provided scoring function and store the result
        res.append({'query_id': q.query_id, 'scores': fn(postings)})

    return res

In [ ]:
print(query_processing(trec_dl_2020.queries_iter(), daat_bm25))

In [ ]:
bm25_results = query_processing(trec_dl_2020.queries_iter(), taat_bm25)
print(bm25_results)

In [ ]:
print(query_processing(trec_dl_2020.queries_iter(), daat_tfidf))

In [ ]:
tfidf_results = query_processing(trec_dl_2020.queries_iter(), taat_tfidf)
print(tfidf_results)

## Evaluation with TREC measures

In [ ]:
for query in list(trec_dl_2020.queries_iter())[:3]:
    print(query)

In [ ]:
for qrel in list(trec_dl_2020.qrels_iter())[:3]:
  print(qrel)

### Run file generation

In [ ]:
def generate_run(results):
    """
    Generate a TREC-formatted run list from query results.

    Args:
        results (list[dict]): A list of query results, where each result contains:
            - `query_id` (int): The ID of the query.
            - `scores` (list[tuple[float, int]]): A list of (score, doc_id) tuples.

    Returns:
        list[str]: A list of strings formatted in TREC run format.
    """

    trec_run_list = []                      # List to store TREC-formatted lines

    for doc_scores in results:              # Iterate over each query result
        rank = 1
        query_id = doc_scores['query_id']
        scores = doc_scores['scores']

        for score, doc_id in scores:
            # Format the result as a TREC-compliant line
            line = f"{query_id} Q0 {doc_id} {rank} {score} GOODFELLAS"
            trec_run_list.append(line)
            rank += 1

    return trec_run_list

# Generate TREC-formatted run lists for BM25 and TF-IDF results
trec_bm25_run_list = generate_run(bm25_results)
trec_tfidf_run_list = generate_run(tfidf_results)

# Write the BM25 run list to a TREC-eval compatible file
with open("trec_eval_bm25_run_file.txt", "w") as f:
    for line in trec_bm25_run_list:
        f.write(line + "\n")

# Write the TF-IDF run list to a separate TREC-eval compatible file
with open("trec_eval_tfidf_run_file.txt", "w") as f:
    for line in trec_tfidf_run_list:
        f.write(line + "\n")

### Qrels file generation

In [ ]:
qrels_file = []     # List to store lines formatted for TREC-Eval qrels

# Iterate over qrels data provided by the TREC DL 2020 dataset
for qrel in trec_dl_2020.qrels_iter():
    # Format the qrel information as per TREC-Eval requirements
    # Format: <query_id> 0 <doc_id> <relevance>
    line = f"{qrel.query_id} 0 {qrel.doc_id} {qrel.relevance}"
    qrels_file.append(line)

# Write the qrels list to a file in TREC-Eval compatible format
with open("trec_eval_qrels_file.txt", "w") as f:
    for line in qrels_file:
        f.write(line + "\n")

### Results

In [ ]:
# Define evaluation measures for the retrieval models
measures = [
    P@5,              # Precision at rank 5
    P(rel=2)@5,       # Precision at rank 5, considering relevance level >= 2
    nDCG@10,          # Normalized Discounted Cumulative Gain at rank 10
    AP,               # Average Precision
    AP(rel=2),        # Average Precision, considering relevance level >= 2
    Bpref,            # Binary preference
    Bpref(rel=2),     # Binary preference, considering relevance level >= 2
    Judged@10         # Fraction of top 10 documents that were judged
]

# Load qrels (ground truth relevance judgments)
qrels = ir_measures.read_trec_qrels('trec_eval_qrels_file.txt')

# Evaluate BM25 results using the defined measures
bm25_run = ir_measures.read_trec_run('trec_eval_bm25_run_file.txt')
bm25_results = ir_measures.calc_aggregate(measures, qrels, bm25_run)

qrels = ir_measures.read_trec_qrels('trec_eval_qrels_file.txt')

# Evaluate TF-IDF results using the same qrels and measures
tfidf_run = ir_measures.read_trec_run('trec_eval_tfidf_run_file.txt')
tfidf_results = ir_measures.calc_aggregate(measures, qrels, tfidf_run)

In [ ]:
import pandas as pd

# Create a DataFrame to compare BM25 and TF-IDF evaluation metrics
# Each column represents the results for a retrieval model
df = pd.DataFrame({
    "BM25": bm25_results,
    "TF-IDF": tfidf_results
})

print(df)

## Comparison with PyTerrier

In [ ]:
from pyterrier.measures import P, nDCG, AP, Judged

# Load the MSMARCO Passage Retrieval dataset
dataset = pt.get_dataset('msmarco_passage')

# Run an experiment comparing TF-IDF and BM25 retrieval models
pt.Experiment(
    [
        # TF-IDF retriever from the Terrier index
        pt.terrier.Retriever.from_dataset('msmarco_passage', 'terrier_stemmed', wmodel='TF_IDF'),
        # BM25 retriever from the Terrier index
        pt.terrier.Retriever.from_dataset('msmarco_passage', 'terrier_stemmed', wmodel='BM25'),
    ],
    dataset.get_topics('test-2020'),                                    # Test topics for the experiment
    dataset.get_qrels('test-2020'),                                     # Ground truth relevance judgments
    eval_metrics=[P@5, P(rel=2)@5, nDCG@10, AP, AP(rel=2), Judged@10],  # Evaluation metrics
)

# Search Engine

In [ ]:
doc_lookup = {doc.doc_id: doc for doc in dataset.docs_iter()}

In [ ]:
class SearchEngine:
    def __init__(self, index, doc_lookup):
        self.index = index
        self.doc_lookup = doc_lookup  # Passa la lookup come parametro

    def search(self):
        user_query = input("Enter your search query: ")
        tokens = set(preprocess(user_query))  # Pre-elabora la query
        term_ids = self.index.get_termids(tokens)  # Ottieni gli ID dei termini
        postings = self.index.get_postings(term_ids)  # Ottieni i postings per gli ID dei termini

        results = new_taat(user_query, postings, self.index.doc)  # Calcolo del ranking

        print("Top results:")
        for score, _,doc_id in results:
            print(doc_id)
            # Recupera i dati del documento dalla lookup
            doc_data = self.doc_lookup.get(doc_id)
            if doc_data:  # Verifica che il documento esista
                print(f"{doc_data.title} (Score: {score:.2f})")
                print(f"Link: {doc_data.url}\n")
            else:
                print(f"Document with ID {doc_id} not found in the dataset.")


# Inizializza il motore di ricerca con la lookup esterna
search_engine = SearchEngine(inv_index, doc_lookup)
search_engine.search()
